### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np
import time
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
#sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

size_scaler = 0.5

#Use same configuration for all experiments, just change some of the parameters

#Define some random paramaters for the creation of the configuration, this will change for each test model
architecture_type = Layers.FullyConnected
#architecture_type = Layers.Convolutional
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size=5, 
                       tournament_size=3, max_similar=3, epochs=5, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=1, binary_selection=True, mutation_ratio=0.8, 
                       similarity_threshold=0.2, more_layers_prob=0.4, verbose_individuals=True, 
                       show_model=True, verbose_training=1)


Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.01, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Get global best model

In [5]:
def recompute_globals_fitness(best_models, size_scaler, problem_type):
    """It is necessary to recompute the fiteness of global models since they have differnt normalization factors"""

    #print("Before normalization")
    #automatic_model_selection.print_best(best_models)
    
    normalize_scores(best_models)
    
    #print("After normalization")
    #automatic_model_selection.print_best(best_models)
    
    compute_fitness(best_models, size_scaler, problem_type)
    
    #print("Recomputed fitness")
    #automatic_model_selection.print_best(best_models)


def normalize_scores(best_models):
    
    pop_size = len(best_models)
    raw_scores = np.zeros((pop_size,))
    
    for i in range(pop_size):
        model = best_models[i]
        raw_scores[i] = model.raw_score
        
    normalization_factor = np.linalg.norm(raw_scores)
    normalized_scores = raw_scores/normalization_factor
    
    for i in range(pop_size):
        model = best_models[i]
        model.normalized_score = raw_scores[i]
    
    
def compute_fitness(best_models, size_scaler, problem_type):
    
    pop_size = len(best_models)
    
    for i in range(pop_size):
        
        round_up_to = 3

        #Round up to the first 3 digits before computing log                                                                                                                                                          
        rounding_scaler = 10**round_up_to
        trainable_count = round(best_models[i].raw_size/rounding_scaler)*rounding_scaler
        size_score = math.log10(trainable_count)

        scaled_score = best_models[i].normalized_score

        #For classification estimate the error which is between 0 and 1                                                                                                                   
        if problem_type == 2:
            metric_score = (1 - scaled_score)*10 #Multiply by 10 to have a better scaling. I still need to find an appropriate scaling                                                
        else:
            metric_score = scaled_score*10 #Multiply by 10 to have a better scaling. I still need to find an appropiate scaling                                                       
    
        metric_scaler = 1-size_scaler
        print("metric_scaler %f"%metric_scaler)
        print("size scaler %f"%size_scaler)
    
        #Scalarization of multiobjective version of the fitness function                                                                                                                  
        best_models[i].fitness = metric_scaler*metric_score + size_scaler*size_score

        

### Test on MNIST

In [6]:
def mnist_test(dhandler_mnist, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    #architecture_type = Layers.FullyConnected
    architecture_type = Layers.Convolutional
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    #input_shape = (784,)
    input_shape = (28,28,1)
    unroll = False

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    total_experiments = 5
    count_experiments = 0
    unroll = True
    """
    #total_experiments = 1
    count_experiments = 0

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = None

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_mnist = MNISTDataHandler()

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape
    config.size_scaler = size_scaler

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                           tournament_size, max_similar, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                           max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                           similarity_threshold=0.2, more_layers_prob=0.8)
    """

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))


        start = time.time()
        best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))


        best.individual_label = count_experiments

        global_best_list.append(best)
        
        """
        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments
        """

        count_experiments =  count_experiments + 1
    
    recompute_globals_fitness(global_best_list, config.size_scaler, config.problem_type)
    
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(experiment_times.sum()))
    logging.info("Global time {}".format(experiment_times.sum()))
    
    return global_best_list, global_best_index, total_experiment_time

### Test on CIFAR10

In [7]:
def cifar_test(dHandler_cifar, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    input_shape = (3072,)
    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0

    scaler = None

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_cifar = CIFAR10DataHandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, 
                           epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, binary_selection=True, 
                           mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler, verbose_data=0)

        best.individual_label = count_experiments

        global_best_list.append(best)

        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments

        count_experiments =  count_experiments + 1
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)
    
    return global_best_list, global_best_index, total_experiment_time

### Test on CMAPSS

In [8]:
def cmapss_test(dhandler_cmaps, input_shape, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 1  #1 for regression, 2 for classification
    output_shape = 1 #If regression applies, number of classes
    input_shape = None

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = MinMaxScaler(feature_range=(-1, 1))

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #cmaps datahandler
    #dhandler_cmaps, input_shape = cmaps_dhandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, 
                           max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, 
                           binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        start = time.time()
        best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))

        best.individual_label = count_experiments

        global_best_list.append(best)

        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments

        count_experiments =  count_experiments + 1
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(total_experiment_time))
    logging.info("Global time {}".format(total_experiment_time))
    
    return global_best_list, global_best_index, total_experiment_time

In [9]:
#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alphas = [0.2]
experiments = 3

global_best_list = []
global_best_index = 0
total_experiment_time = []
total_experiment_time = 0
avg_experiment_time = 0

dHandler_mnist = MNISTDataHandler()

for size_scaler in alphas:

    print("Running for alpha={}".format(size_scaler))
    
    global_best_list, global_best_index, total_experiment_time = mnist_test(dHandler_mnist, 
                                                                            size_scaler=size_scaler, 
                                                                            total_experiments=experiments)
    
    avg_experiment_time = total_experiment_time/experiments
    
    print("Total experiment time {}".format(total_experiment_time))
    print("Avg experiment time {}".format(avg_experiment_time))
    
    save_best_models(global_best_list, global_best_index, 
                     'best_models/mnist/scalarized/alpha{}/'.format(size_scaler), input_shape=input_shape, 
                     data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)
    

Running for alpha=0.2
Launching experiment 1
Generating layer limits according to dataset
(28, 28, 1)
Loading data. Cross-Validation ratio 0
28
{}
Previous layer
None
Layer limits
{'max_neuron_multiplier': 128, 'max_filter_size_multiplier': 64, 'max_kernel_size_multiplier': 14, 'max_filter_stride': 3, 'max_pooling_exponent': 6, 'max_dropout': 0.7}
Previous Layer
None
generating convolutional layer
filter size 80
previous size 14
max kernel size 7 
max kernel size multiplier 7 
Conv layer generated
[<Layers.Convolutional: 2>, 0, 1, 80, 5, 3, 0, 0, 0, 0]
{<Layers.Convolutional: 2>: 1}
Previous layer
{<Layers.Convolutional: 2>: 1}
Previous layer
{<Layers.Convolutional: 2>: 1}
{}
Previous layer
None
Layer limits
{'max_neuron_multiplier': 128, 'max_filter_size_multiplier': 64, 'max_kernel_size_multiplier': 14, 'max_filter_stride': 3, 'max_pooling_exponent': 6, 'max_dropout': 0.7}
Previous Layer
None
generating convolutional layer
filter size 504
previous size 14
max kernel size 7 
max kerne

KeyError: 8

In [ ]:
""""
dHandler_cifar = CIFAR10DataHandler()

global_best_list, global_best_index = cifar_test(dHandler_cifar, size_scaler=size_scaler, total_experiments=1)
"""

In [ ]:
"""
dhandler_cmaps, input_shape = cmaps_dhandler()

global_best_list, global_best_index = cmapss_test(dhandler_cmaps, input_shape, size_scaler=size_scaler, total_experiments=1)
"""

In [ ]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/scalarized_version/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)
"""

In [ ]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/version2'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)
"""

In [ ]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)
"""